# LRT Request Analysis

## Functionalities
- Analysis of RPCs and queries of LRT requests.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
```
`*_service*.tar.gz` and `apigateway*.tar.gz` tarballs contain RPC log files named `calls.log` and database query log files named `queries.log`. A tarball `loadgen.tar.gz` contains a request log file named `loadgen.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-11-11-16-06-13"

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from parsers.loadgen_parser import LoadgenParser
from parsers.query_parser import QueryParser
from parsers.rpc_parser import RPCParser
from utils.utils import *

## Request Log Parsing

In [ ]:
# Build data frame
requests = pd.concat(
    [pd.DataFrame.from_dict(LoadgenParser(logfile).parse()) for logfile in get_loadgen_logfiles(EXPERIMENT_DIRNAME)],
    ignore_index=True)

## RPC Log Parsing

In [ ]:
# Build data frame
rpc = pd.concat(
        [pd.DataFrame.from_dict(RPCParser(logfile).parse()) for logfile in get_rpc_logfiles(EXPERIMENT_DIRNAME)], 
        ignore_index=True)

## Query Log Parsing

In [ ]:
# Build data frame
query = pd.concat(
        [pd.DataFrame.from_dict(QueryParser(logfile).parse()) for logfile in get_query_logfiles(EXPERIMENT_DIRNAME)],
        ignore_index=True)

## Analysis of LRT Requests

In [ ]:
# Analysis of requests with latency over P99.9
lrt_requests = requests[(requests["latency"] > requests[requests["status_code"] == 200]["latency"].quantile(0.999))]
for lrt_request in lrt_requests.to_dict("records"):
    print(lrt_request)
    print("Request ID: %s" % lrt_request["request_id"])
    print("  Type: %s" % lrt_request["type"])
    print("  RPCs:")
    for lrt_request_rpc in rpc[(rpc["request_id"] == lrt_request["request_id"])].to_dict("records"):
        print("    %s - %s" % (lrt_request_rpc["function"], lrt_request_rpc["latency"]))
    print("  Queries:")
    for lrt_request_query in query[(query["request_id"] == lrt_request["request_id"])].to_dict("records"):
        print("    %s - %s" % (lrt_request_query["dbname"] + ":" + lrt_request_query["type"],
                lrt_request_query["latency"]))